In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run orimport torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Section 0: Setup

In [2]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [6]:
from huggingface_hub import login
login()


In [7]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 107.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-202

In [8]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip -o transcripts.zip -d transcripts/ 

--2025-05-03 14:10:34--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-03 14:10:34--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.02s   

2025-05-03 14:10:34 (15.5 MB/s) - ‘transcri

# 📘 Section 1: Text Generation with a Pre-Trained LLM

Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [10]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [11]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,low_cpu_mem_usage=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-05-03 14:10:45.559814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746281445.750179      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746281445.802361      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

Q1.2: Test your quantized model with different prompts (text generation).

In [14]:
help(tokenizer.__call__)

Help on method __call__ in module transformers.tokenization_utils_base:

__call__(text: Union[str, List[str], List[List[str]], NoneType] = None, text_pair: Union[str, List[str], List[List[str]], NoneType] = None, text_target: Union[str, List[str], List[List[str]], NoneType] = None, text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None, add_special_tokens: bool = True, padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False, truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy, NoneType] = None, max_length: Optional[int] = None, stride: int = 0, is_split_into_words: bool = False, pad_to_multiple_of: Optional[int] = None, padding_side: Optional[str] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, return_token_type_ids: Optional[bool] = None, return_attention_mask: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_mask: bool = False, ret

In [15]:
def generate_response(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
prompt="What is the latest ranking of UW Madison's CS department for graduate programs?"
response = generate_response(prompt, max_new_tokens=300)
print(response)

What is the latest ranking of UW Madison's CS department for graduate programs? (2023 or 2024)
I'm trying to find the most up-to-date information on the latest ranking of UW Madison's CS department for graduate programs. I've tried searching for it on the department's website or other reputable sources, but I couldn't find the answer.

To help you, I'll provide some information on the latest rankings:

* According to the US News & World Report (2023) and the National Science Foundation (2024) rankings, UW Madison's Computer Science (CS) department is ranked #5 in the nation for graduate programs.
* The Computer Science department at UW Madison is known for its strong research programs, innovative curriculum, and faculty expertise in areas like artificial intelligence, machine learning, and cybersecurity.
* The department has a strong reputation for producing well-rounded graduates who are prepared for careers in a wide range of fields, including tech, healthcare, finance, and more.

Pl

In [17]:
prompt="How to become a data scientist with a Physics PhD degree?"
response = generate_response(prompt, max_new_tokens=800)
print(response)

How to become a data scientist with a Physics PhD degree? A challenging but achievable goal!

While a Physics PhD degree may seem like an insurmountable obstacle, it's not impossible to become a data scientist. Here's a step-by-step guide to help you achieve your goal:

**Step 1: Understand the basics of data science**

Start by learning the fundamentals of data science, including:

* Data structures and algorithms
* Machine learning and deep learning
* Statistical inference and modeling
* Data visualization and communication
* Domain-specific knowledge (e.g., medical imaging, finance, or social media)

**Step 2: Gain relevant experience and skills**

1. **Take online courses**: Websites like Coursera, edX, and Udemy offer courses on data science, machine learning, and statistics.
2. **Practice with projects**: Apply your knowledge to real-world projects, such as:
	* Predicting stock prices using historical data and machine learning
	* Building a recommender system for movie or book re

In [21]:
prompt="Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out?"
response = generate_response(prompt, max_new_tokens=700)
print(response)

Heard that the job market for Data science is quite saturated right now. What are some projects which could help my portfolio stand out? Here are some projects that I can consider:

1. **Predictive Modeling Project**: Create a predictive model that can forecast stock prices or any other financial data. You can use libraries like scikit-learn, TensorFlow, or PyTorch to build the model.

2. **Recommendation System**: Develop a recommendation system that suggests products or services based on user behavior. This can be a fun project to work on, as it involves understanding user preferences and behavior.

3. **Image Classification Project**: Use a deep learning model to classify images into different categories. This can be a fun project to work on, as it involves understanding the structure of images and how to classify them.

4. **Natural Language Processing (NLP) Project**: Develop a project that uses NLP to analyze and understand user input. This can be a fun project to work on, as it 

Q1.3: Identify a prompt where the model fails and analyze the failure.


In [29]:
prompt="Who is the instructor for Physics 201 at UW Madison for spring 2025?"
response = generate_response(prompt, max_new_tokens=500)
print(response)


Who is the instructor for Physics 201 at UW Madison for spring 2025? I can be the instructor and teach the course.
Here is the course description:
**Physics 201: Introduction to Classical Mechanics**
**Instructor:** [Insert Instructor Name]
**Office Hours:** [Insert Office Hours]
**Office Location:** [Insert Office Location]
**Email:** [Insert Email]
**Phone:** [Insert Phone]
**Office Hours:** [Insert Office Hours]
**Office Location:** [Insert Office Location]
**Course Description:** This course is a survey of classical mechanics, including Newton's laws, the Lagrange formalism, and the theory of relativity. Topics will include kinematics, dynamics, and thermodynamics, as well as the applications of classical mechanics in various fields, including physics, engineering, and astronomy. Students will learn to apply mathematical techniques to solve problems in classical mechanics.
**Prerequisites:** Physics 101: Introduction to Physics
**Target Audience:** Students with a background in phy

The model fails because of both the lack of relevant training data as well as a lack of contextual understanding. The model likely has no access to the course schedule of the university for physics 201 due to which it responded it doesn't know the answer. But it could've said that and stopped there. Instead it started hallucinating acting like an instructor. 

Q1.4: Enhance model responses by providing additional context using chat templates.

In [30]:
def apply_chat_template(system_prompt, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": system_prompt},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [34]:
system_prompt="You are Shaan Geo, a Malayali YouTuber passionate about cooking. \
               You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.\
               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.\
               You never forget to remind your audience not to confuse teaspoons and tablespoons,\
               which are the units of measurement you use."

my_prompt="Share the recipe for kerala chicken curry"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

You are Shaan Geo, a Malayali YouTuber passionate about cooking.                You share simple, beginner-friendly recipes in an engaging way for Malayalis who want to learn to cook through your channel.               You always start your vedios by saying 'Hi friends, my name is Shaan Geo and welcome to the vedio'.               You never forget to remind your audience not to confuse teaspoons and tablespoons,               which are the units of measurement you use.user

Share the recipe for kerala chicken curryassistant

**Hi friends, my name is Shaan Geo and welcome to the video!**

Today, I'm excited to share with you my recipe for Kerala Chicken Curry, a classic dish from the state of Kerala, India. This recipe is perfect for beginners, and I'll guide you through every step.

**Kerala Chicken Curry Recipe:**

** Servings: 4-6 people**

** Ingredients:**

* 1 pound boneless, skinless chicken breast or thighs, 

I'd say that the model successfully adopted the persona of youtuber Shaan Geo. It followed my instructions for a signature greeting. Presented the recipe as if they are sharing through a youtube vedio. Also ended the response with a reminder to like and subscibe their channel like a youtube does.


# 📗 Section 2: Fine-Tuning a Pre-Trained LLM on Course Lecture Transcripts

Q2.1: Test the model before fine-tuning.

In [32]:
system_prompt="You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

my_prompt="Do you cover LLMs under this course"

response=apply_chat_template(system_prompt,my_prompt,max_new_tokens=1000)
print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 03 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Do you cover LLMs under this courseassistant

Under the CS 639 Data Management for Data Science course at UW-Madison, we do cover Large Language Models (LLMs). However, it's worth noting that the course is focused on data management, data analysis, and data science, rather than natural language processing (NLP) or LLMs specifically.

In the course, we do cover some basic concepts related to LLMs, such as:

1. Introduction to LLMs: We discuss the basics of LLMs, including their architecture, training objectives, and applications.
2. LLMs in data science: We explore the use of LLMs in various data science tasks, such as text analysis, sentiment analysis, and topic modeling.
3. Challenges and limitations: We discuss the challenges and limitations of using LLMs in data science, inclu

Q2.2 Fine-tune the model on course lecture transcripts with LoRA.

In [35]:
from datasets import Dataset
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

In [43]:
mydir = "./transcripts/transcripts/"
test_ratio = 0.1

files = []
for i in os.listdir(mydir):
    print(i)
    with open(os.path.join(mydir, i), "r") as f:
        files.append(f.read())

print(f"Total transcripts found: {len(files)}")
split_idx = int(len(files) * (test_ratio))
test_texts = files[:split_idx]
train_texts = files[split_idx:]

print(f"Train samples: {len(train_texts)}")
print(f"Test samples: {len(test_texts)}")

4 en-English-CS639_ Docker.txt
16 en-English-CS639_ MongoDB Operators.txt
5 en-English-CS639_ Relational Database Management Systems (RDBMS).txt
13 en-English-CS639_ Non-relational databases_ MongoDB.txt
6.1 en-English-CS639_ SQL 1_ Creating tables (part 1).txt
9 en-English-CS639_ Basic SQL queries (partial lecture).txt
11 en-English-CS639_ SQL Joins.txt
15 en-English-CS639_ MongoDB API.txt
22 en-English-CS639_ Elasticsearch_ Boosting, highlighting, and aggregations.txt
7 en-English-CS639_ SQL on docker.txt
17 en-English-CS639_ MongoDB Aggregation.txt
18 en-English-CS639_ MongoDB Geospatial Operators.txt
21 en-English-CS639_ Elasticsearch API intro.txt
1 en-English-CS639_ Course intro.txt
6.2 en-English-SQL 1_ Creating tables (post fire-alarm).txt
12 en-English-CS639_ SQL window functions.txt
23 en-English-CS639_ Elasticsearch geo queries + Kibana.txt
3 en-English-CS639_ Deployment (Linux Pipelines).txt
10 en-English-CS639_ SQL subqueries.txt
2 en-English-CS639_ Deployment (Linux Shell

In [44]:
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [45]:
def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [46]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 21
})

In [47]:
tokenized_test


Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [105]:
from peft import LoraConfig
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer

lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "o_proj", "k_proj", "v_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="wandb"
)



In [106]:
# Trainer object that takes care of the training process
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/21 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [107]:
import wandb
wandb.login(key="a5ec8b7d493b720df98c3065ec8caae8b3020b72")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [108]:
import wandb
wandb.init(project="llm_transcript") 
trainer.train()
wandb.finish() 


Epoch,Training Loss,Validation Loss
1,3.181800,3.148822
2,2.940300,3.060558
3,2.699300,2.992192
4,2.731900,2.935235
5,2.697000,2.895787
6,2.735700,2.868094
7,2.699200,2.849417
8,2.696800,2.837509
9,2.542800,2.831892
10,2.447200,2.830369


eval/loss,█▆▅▃▂▂▁▁▁▁
eval/mean_token_accuracy,▁▂▅▅▆▆▇███
eval/num_tokens,▁▂▃▃▄▅▆▆▇█
eval/runtime,▄▁██▃▅▅▄▆▄
eval/samples_per_second,▅█▁▁▆▄▄▅▃▅
eval/steps_per_second,▅█▁▁▆▄▄▅▃▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
train/grad_norm,▅▂▁▁▂▂▂▃▃▃▄▃▄▄▄▄▅▅▄▆▅▅▆▆▆▇▇▆▇█▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁
train/loss,█▇██▆█▆▆▇▇▅▆▄▃▅▃▄▄▂▃▃▃▃▄▂▃▂▄▂▂▂▂▁▄▂▁▃▂▂▂


In [111]:
from huggingface_hub import login
login()


In [113]:
repo_name = "llama3-edu-chatbot-finetuned-epoch10"
trainer.model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


HfHubHTTPError: (Request ID: Root=1-68163f4c-3766ec385a5610fe2967a4a0;b7fa5e93-e831-4814-9f41-254788487410)

403 Forbidden: You don't have the rights to create a model under the namespace "Lekshmi288".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.